In [3]:
import numpy as np
from PIL import Image
from noise import pnoise3
import math
import os

In [4]:
!pip install noise numpy Pillow

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import numpy as np
from PIL import Image
from noise import pnoise3
import math
import os

# --------------------------------------------------
# 1) Noise Functions
# --------------------------------------------------

def sine_noise(x, y, frequency=10.0, amplitude=1.0):
    """
    Simple sine wave.
    """
    return amplitude * np.sin(frequency * x)

def square_noise(x, y, frequency=10.0, amplitude=1.0):
    """
    Simple square wave derived from sine.
    """
    return amplitude * np.sign(np.sin(frequency * x))

def fbm_noise(x, y, z=0, octaves=4, persistence=0.5, lacunarity=2.0):
    """
    Fractal Brownian Motion noise using Perlin noise internally (pnoise3).
    """
    value = 0.0
    amp = 1.0
    freq = 1.0
    for _ in range(octaves):
        value += amp * pnoise3(x * freq, y * freq, z * freq)
        amp *= persistence
        freq *= lacunarity
    return value

# Alias "fractal_noise" to fbm_noise if you want a separate name:
fractal_noise = fbm_noise

def perlin_noise(x, y, z=0, scale=1.0):
    """
    Basic Perlin noise.
    """
    return pnoise3(x * scale, y * scale, z * scale)

def beckmann_noise(x, y, alpha=0.5):
    """
    Beckmann microfacet distribution function (D-term).
    """
    tan_theta_h = np.sqrt(x**2 + y**2) / alpha
    cos_theta_h = 1.0 / np.sqrt(1.0 + tan_theta_h**2)
    D = np.exp(-(tan_theta_h**2)) / (math.pi * alpha**2 * cos_theta_h**4)
    return D

def ggx_noise(x, y, alpha=0.5):
    """
    GGX (Trowbridge-Reitz) microfacet distribution function (D-term).
    """
    tan2_theta_h = (x**2 + y**2) / alpha**2
    cos_theta_h = 1.0 / np.sqrt(1.0 + tan2_theta_h)
    cos2_theta_h = cos_theta_h**2
    D = alpha**2 / (math.pi * ((cos2_theta_h * (alpha**2 - 1.0) + 1.0)**2))
    return D

def blinn_noise(x, y, n=20):
    """
    Blinn-Phong microfacet distribution function (D-term).
    """
    tan_theta_h = math.sqrt(x**2 + y**2)
    cos_theta_h = 1.0 / math.sqrt(1.0 + tan_theta_h**2)
    D = (n + 2) / (2.0 * math.pi) * (cos_theta_h**n)
    return D

def gabor_noise(x, y, frequency=5.0, sigma=0.2):
    """
    Simplified Gabor-like filter approach (demonstration).
    This is not a full Gabor library implementation!
    """
    # Gabor kernel: a complex exponential modulated by a Gaussian
    # We'll just do real part for demonstration
    gauss = np.exp(-((x**2 + y**2) / (2 * sigma**2)))
    wave = np.cos(2.0 * math.pi * frequency * x)
    return gauss * wave

def simple_3d_noise(x, y, t=0.0, scale=0.5):
    """
    Use a time or 'z' dimension to create a 3D noise effect (pnoise3).
    """
    return pnoise3(x * scale, y * scale, t)

def mandelbrot_noise(x, y, L=10.0, D_f=1.5, gamma=1.2, M=10, N_max=10, rng=None):
    """
    Ausloos-Berman Weierstrass-Mandelbrot fractal noise function.
    Generates a fractal height value at coordinates (x, y) using the formula:

        y(x) = L^(4 - 2*D_f) * ln(gamma) * ∑[n=1 to N_max] ∑[m=1 to M] [
                    gamma^( (n-1)*(2(D_f - 2)) ) *
                    cos( 2π * gamma^(n-1) * x/L - cos(πm/M) + φ_mn )
                  + same for y
                  ]

    Parameters:
        x, y (float): Spatial coordinates for evaluation.
        L (float): Length scale of the noise.
        D_f (float): Fractal dimension (affects scaling).
        gamma (float): Frequency scaling factor.
        M (int): Number of angular divisions.
        N_max (int): Maximum frequency index.
        rng (numpy.random.Generator): Optional random generator for phases.
    Returns:
        float: Fractal noise value at (x, y).
    """
    if rng is None:
        rng = np.random.default_rng()

    # Precompute scaling factors
    log_gamma = math.log(gamma)
    scale_factor = (L**(4 - 2.0 * D_f)) * log_gamma
    noise_value = 0.0

    for n in range(1, N_max + 1):
        freq_scale = gamma**(n - 1)
        amp_scale = freq_scale**(2.0 * (D_f - 2.0))
        # Random phase array for each m
        phi_mn = rng.uniform(0, 2.0 * math.pi, M)

        for m in range(1, M + 1):
            cos_part = math.cos(math.pi * m / M)
            # Use the same random phase for x & y in each m term
            phase = phi_mn[m - 1]

            # x-direction
            arg_x = (2.0 * math.pi * freq_scale * x / L) - cos_part + phase
            # y-direction
            arg_y = (2.0 * math.pi * freq_scale * y / L) - cos_part + phase

            noise_value += amp_scale * (math.cos(arg_x) + math.cos(arg_y))

    return scale_factor * noise_value

# --------------------------------------------------
# 2) Displacement Map Generator
# --------------------------------------------------

def generate_displacement_map(
    noise_func,
    map_size=256,
    noise_params=None,
    x_range=(-1, 1),
    y_range=(-1, 1),
    normalize=True
):
    """
    Create a 2D displacement map using a given noise function.
    :param noise_func: callable, e.g., sine_noise, fbm_noise, etc.
    :param map_size: resolution of the map, e.g., 256 => 256x256
    :param noise_params: dict of extra parameters for the noise function
    :param x_range: tuple (min_x, max_x)
    :param y_range: tuple (min_y, max_y)
    :param normalize: bool, if True normalize to [0,1]
    :return: a 2D numpy array
    """
    if noise_params is None:
        noise_params = {}

    min_x, max_x = x_range
    min_y, max_y = y_range

    xs = np.linspace(min_x, max_x, map_size)
    ys = np.linspace(min_y, max_y, map_size)

    displacement = np.zeros((map_size, map_size), dtype=np.float32)
    
    for i in range(map_size):
        for j in range(map_size):
            x_val = xs[i]
            y_val = ys[j]
            # Compute noise
            displacement[i, j] = noise_func(x_val, y_val, **noise_params)

    if normalize:
        dmin, dmax = displacement.min(), displacement.max()
        if dmax > dmin:
            displacement = (displacement - dmin) / (dmax - dmin)
        else:
            displacement[:] = 0.0  # If all the same value, fill with 0

    return displacement

def save_displacement_map(displacement, filename="displacement.png"):
    """
    Save a 2D numpy array (values in [0,1]) as a grayscale PNG.
    """
    # Convert to 8-bit grayscale
    displacement_8u = (displacement * 255).astype(np.uint8)
    img = Image.fromarray(displacement_8u, mode='L')
    img.save(filename)
    print(f"Saved displacement map to {filename}")

# --------------------------------------------------
# 3) Example Usage: Generate and Save Each Noise
# --------------------------------------------------

if __name__ == "__main__":
    # Output folder
    os.makedirs("displacement_maps", exist_ok=True)

    # A dictionary mapping function names to (function, default_parameters)
    noise_variants = {
        "sine":       (sine_noise,       {"frequency": 5.0,  "amplitude": 1.0}),
        "square":     (square_noise,     {"frequency": 5.0,  "amplitude": 1.0}),
        "fractal":    (fractal_noise,    {"octaves": 4, "persistence": 0.5, "lacunarity": 2.0}),
        "perlin":     (perlin_noise,     {"scale": 0.5}),
        "fbm":        (fbm_noise,        {"octaves": 5, "persistence": 0.4, "lacunarity": 2.3}),
        "ggx":        (ggx_noise,        {"alpha": 0.4}),
        "beckmann":   (beckmann_noise,   {"alpha": 0.3}),
        "blinn":      (blinn_noise,      {"n": 20}),
        "gabor":      (gabor_noise,      {"frequency": 4.0, "sigma": 0.3}),
        "3D":         (simple_3d_noise,  {"t": 0.5, "scale": 0.4}),
        "mandelbrot": (mandelbrot_noise, {"L": 10.0, "D_f": 1.5, "gamma": 1.2, "M": 10, "N_max": 10}),
    }

    # Generate and save each map
    for name, (func, params) in noise_variants.items():
        disp = generate_displacement_map(
            noise_func=func,
            map_size=256,       # resolution
            noise_params=params,
            x_range=(-1, 1),
            y_range=(-1, 1),
            normalize=True
        )
        fname = f"displacement_maps/{name}_displacement.png"
        save_displacement_map(disp, fname)


Saved displacement map to displacement_maps/sine_displacement.png
Saved displacement map to displacement_maps/square_displacement.png
Saved displacement map to displacement_maps/fractal_displacement.png
Saved displacement map to displacement_maps/perlin_displacement.png
Saved displacement map to displacement_maps/fbm_displacement.png
Saved displacement map to displacement_maps/ggx_displacement.png
Saved displacement map to displacement_maps/beckmann_displacement.png
Saved displacement map to displacement_maps/blinn_displacement.png
Saved displacement map to displacement_maps/gabor_displacement.png
Saved displacement map to displacement_maps/3D_displacement.png
Saved displacement map to displacement_maps/mandelbrot_displacement.png
